# 01 Import Libraries and Dataframes

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#Import dataframes
path = r'/Users/Nick/Documents/CareerFoundry/Immersion 4/Instacart Basket Analysis 11.11.21/02 Data'
#Orders & Products
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged_agg_flag.pkl'))
#Customers
customers = pd.read_csv(os.path.join(path, 'Original Data', 'customers.csv'), index_col = False)

In [3]:
pd.options.display.max_columns = 40

In [4]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_item_price,spending_flag,median_days_since_prior_order,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy day,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [5]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
customers.shape

(206209, 10)

# 02 Data Wrangling

In [8]:
#Drop irrelevant columns
customers = customers.drop(columns = ['First Name', 'Surnam', 'date_joined'])

In [9]:
customers.head()

,user_id,Gender,STATE,Age,n_dependants,fam_status,income
0,26711,Female,Missouri,48,3,married,165665
1,33890,Female,New Mexico,36,0,single,59285
2,65803,Male,Idaho,35,2,married,99568
3,125935,Female,Iowa,40,0,single,42049
4,130797,Female,Maryland,26,1,married,40374


In [10]:
#Rename columns
customers.rename(columns = {'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age'}, inplace = True)

In [11]:
customers.head()

,user_id,gender,state,age,n_dependants,fam_status,income
0,26711,Female,Missouri,48,3,married,165665
1,33890,Female,New Mexico,36,0,single,59285
2,65803,Male,Idaho,35,2,married,99568
3,125935,Female,Iowa,40,0,single,42049
4,130797,Female,Maryland,26,1,married,40374


# 03 Data Quality and Consistency Checks

In [12]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   gender        206209 non-null  object
 2   state         206209 non-null  object
 3   age           206209 non-null  int64 
 4   n_dependants  206209 non-null  int64 
 5   fam_status    206209 non-null  object
 6   income        206209 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 11.0+ MB


In [13]:
ords_prods_merge['user_id'].dtype

dtype('int64')

In [15]:
#Check for mixed data type
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len(customers[weird] > 0):
        print (col)

In [16]:
#Check for null values
customers.isnull().sum()

user_id         0
gender          0
state           0
age             0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [17]:
#Check for duplicates
customers_dups = customers[customers.duplicated()]

In [18]:
customers_dups

,user_id,gender,state,age,n_dependants,fam_status,income


#### No mixed data types, missing values, or duplicates

# 04 Combine Dataframes

In [19]:
#Drop old merge indicator 
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])
#Combine Customers with Orders & Products
ords_prods_cust_merge = ords_prods_merge.merge(customers, on = ['user_id'], indicator = True)

In [20]:
ords_prods_cust_merge.shape

(32404859, 30)

In [21]:
#Check merge success
ords_prods_cust_merge['_merge'].value_counts(dropna = False)

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [22]:
#Drop merge indicator
ords_prods_cust_merge = ords_prods_cust_merge.drop(columns = ['_merge'])

In [23]:
ords_prods_cust_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_item_price,spending_flag,median_days_since_prior_order,order_frequency_flag,gender,state,age,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy day,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy day,Less busy day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423


# 05 Export dataframe

In [24]:
ords_prods_cust_merge.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_customers.pkl'))